In [54]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from fredapi import Fred
import sqlite3
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.dates as mdates
from pandas.tseries.offsets import Day
from decimal import Decimal
import requests
import seaborn as sns
color_pal = sns.color_palette()
from pandas.tseries.offsets import BDay
from multiprocessing import Pool
from tqdm import tqdm
from IPython.display import display
from IPython.display import HTML, display

In [55]:
# Function to get the next trading day
def get_next_trading_day():
    today = datetime.today()
    next_trading_day = today + BDay(1)
    return next_trading_day

In [56]:
# Parameters
ticker = "^GSPC"
start_date = "1950-01-01"
end_date = get_next_trading_day().strftime('%Y-%m-%d')

In [57]:
end_date

'2024-11-19'

In [58]:
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [59]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
1950-01-03 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000


In [60]:
data.columns = data.columns.droplevel(1)  # Drop the second level (ticker)
print(data.head())

Price                      Adj Close  Close   High    Low   Open   Volume
Date                                                                     
1950-01-03 00:00:00+00:00      16.66  16.66  16.66  16.66  16.66  1260000
1950-01-04 00:00:00+00:00      16.85  16.85  16.85  16.85  16.85  1890000
1950-01-05 00:00:00+00:00      16.93  16.93  16.93  16.93  16.93  2550000
1950-01-06 00:00:00+00:00      16.98  16.98  16.98  16.98  16.98  2010000
1950-01-09 00:00:00+00:00      17.08  17.08  17.08  17.08  17.08  2520000


In [61]:
# Calculate daily returns
data['Index_Returns'] = data['Adj Close'].pct_change()
data.dropna(inplace=True)

In [62]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns
Date,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927
...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050


In [63]:
# Check for NaNs in 'Index_Returns' and drop them
returns = data['Index_Returns'].dropna()

In [64]:
returns

Date
1950-01-04 00:00:00+00:00    0.011405
1950-01-05 00:00:00+00:00    0.004748
1950-01-06 00:00:00+00:00    0.002953
1950-01-09 00:00:00+00:00    0.005889
1950-01-10 00:00:00+00:00   -0.002927
                               ...   
2024-11-12 00:00:00+00:00   -0.002893
2024-11-13 00:00:00+00:00    0.000232
2024-11-14 00:00:00+00:00   -0.006050
2024-11-15 00:00:00+00:00   -0.013203
2024-11-18 00:00:00+00:00    0.004173
Name: Index_Returns, Length: 18841, dtype: float64

In [65]:
# Fit Markov Switching Model
model = MarkovRegression(returns, k_regimes=2, trend='c', switching_variance=True)
result = model.fit()
print(result.summary())

/opt/anaconda3/envs/msm_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                        Markov Switching Model Results                        
Dep. Variable:          Index_Returns   No. Observations:                18841
Model:               MarkovRegression   Log Likelihood               63446.461
Date:                Mon, 18 Nov 2024   AIC                        -126880.922
Time:                        15:36:15   BIC                        -126833.859
Sample:                             0   HQIC                       -126865.480
                              - 18841                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   5.65e-05     11.507      0.000       0.001       0.001
sigma2          4e-05   8.07e-07     49.569      0.0

In [66]:
# Add regime to the data
data.loc[returns.index, 'Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)

/var/folders/7f/vv91r0d95232m0z1hmgmv6xw0000gn/T/ipykernel_20431/2742928058.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[returns.index, 'Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)


In [67]:
# # Create a scrollable display
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     html_output = data.to_html()

# display_html = f"""
# <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc; padding: 10px;">
#     {html_output}
# </div>
# """
# display(HTML(display_html))

In [68]:
data['Vol_Regime']

Date
1950-01-04 00:00:00+00:00    0
1950-01-05 00:00:00+00:00    0
1950-01-06 00:00:00+00:00    0
1950-01-09 00:00:00+00:00    0
1950-01-10 00:00:00+00:00    0
                            ..
2024-11-12 00:00:00+00:00    0
2024-11-13 00:00:00+00:00    0
2024-11-14 00:00:00+00:00    0
2024-11-15 00:00:00+00:00    0
2024-11-18 00:00:00+00:00    0
Name: Vol_Regime, Length: 18841, dtype: int64

In [69]:
# Extract smoothed probabilities and last known state probabilities
smoothed_probs = result.smoothed_marginal_probabilities

In [70]:
smoothed_probs

,0,1
Date,,
1950-01-04 00:00:00+00:00,0.964036,0.035964
1950-01-05 00:00:00+00:00,0.977873,0.022127
1950-01-06 00:00:00+00:00,0.983236,0.016764
1950-01-09 00:00:00+00:00,0.983472,0.016528
1950-01-10 00:00:00+00:00,0.980390,0.019610
...,...,...
2024-11-12 00:00:00+00:00,0.881006,0.118994
2024-11-13 00:00:00+00:00,0.889884,0.110116
2024-11-14 00:00:00+00:00,0.885290,0.114710


In [71]:
last_probs = smoothed_probs.iloc[-1].values

In [72]:
last_probs

array([0.87964693, 0.12035307])

In [73]:
# Extract transition probabilities from the model parameters
params = result.params
p_00 = params['p[0->0]']
p_10 = params['p[1->0]']
p_01 = 1 - p_00
p_11 = 1 - p_10

In [74]:
# Construct the transition matrix
transition_matrix = np.array([
    [p_00, p_01],
    [p_10, p_11]
])

In [75]:
transition_matrix

array([[0.98805245, 0.01194755],
       [0.03734518, 0.96265482]])

In [76]:
# Update state probabilities to predict the next day's regime
state_probs = np.dot(last_probs, transition_matrix)

In [77]:
state_probs

array([0.87363191, 0.12636809])

In [78]:
# Determine the most likely regime at t+1
regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]
predicted_most_likely_regime = regime_labels[np.argmax(state_probs)]


In [79]:
predicted_most_likely_regime

0

In [80]:
# Assuming end_date is already defined as a string in the format 'YYYY-MM-DD'
end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Convert to datetime object
predicted_date = (end_date + timedelta(days=0)).strftime('%Y-%m-%d')  # Increment and convert back to string

In [81]:
predicted_date

'2024-11-19'

In [82]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime
Date,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0
...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232,0
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050,0


In [83]:
def triangular_moving_average(series, n):
    smoothed_series = series.rolling(window=n//2, min_periods=1).mean()
    smoothed_series = smoothed_series.rolling(window=n//2, min_periods=1).mean()
    return smoothed_series

In [84]:
# Calculate 250-day triangular moving average
data['250_TMA'] = triangular_moving_average(data['Adj Close'], 250)

In [85]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime,250_TMA
Date,,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0,16.850000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0,16.870000
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0,16.886667
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0,16.905000
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0,16.918800
...,...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0,5270.483643
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232,0,5275.503745
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050,0,5280.514303


In [86]:
# Define the four market regimes for 250 TMA
conditions = [
    (data['Vol_Regime'] == 1) & (data['Adj Close'] < data['250_TMA']),
    (data['Vol_Regime'] == 1) & (data['Adj Close'] >= data['250_TMA']),
    (data['Vol_Regime'] == 0) & (data['Adj Close'] < data['250_TMA']),
    (data['Vol_Regime'] == 0) & (data['Adj Close'] >= data['250_TMA']),
]
choices = [
    'Bearish High Variance',
    'Bullish High Variance',
    'Bearish Low Variance',
    'Bullish Low Variance'
]

In [87]:
# Specify a default value that matches the data type of choices
data['Market_Regime'] = np.select(conditions, choices, default='Unknown')

In [88]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime,250_TMA,Market_Regime
Date,,,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0,16.850000,Bullish Low Variance
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0,16.870000,Bullish Low Variance
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0,16.886667,Bullish Low Variance
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0,16.905000,Bullish Low Variance
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0,16.918800,Bullish Low Variance
...,...,...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0,5270.483643,Bullish Low Variance
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232,0,5275.503745,Bullish Low Variance
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050,0,5280.514303,Bullish Low Variance


In [89]:
# Calculate 30-day and 60-day Triangular Moving Averages and shift by 1 day
data['30_TMA'] = triangular_moving_average(data['Adj Close'], 30).shift(1)
data['60_TMA'] = triangular_moving_average(data['Adj Close'], 60).shift(1)

# Define 30-Day and 60-Day Indicators
data['30_Day_Indicator'] = np.where(data['Adj Close'] > data['30_TMA'], 'Bullish', 'Bearish')
data['60_Day_Indicator'] = np.where(data['Adj Close'] > data['60_TMA'], 'Bullish', 'Bearish')

In [90]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime,250_TMA,Market_Regime,30_TMA,60_TMA,30_Day_Indicator,60_Day_Indicator
Date,,,,,,,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0,16.850000,Bullish Low Variance,NaN,NaN,Bearish,Bearish
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0,16.870000,Bullish Low Variance,16.850000,16.850000,Bullish,Bullish
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0,16.886667,Bullish Low Variance,16.870000,16.870000,Bullish,Bullish
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0,16.905000,Bullish Low Variance,16.886667,16.886667,Bullish,Bullish
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0,16.918800,Bullish Low Variance,16.905000,16.905000,Bullish,Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0,5270.483643,Bullish Low Variance,5815.639277,5717.656303,Bullish,Bullish
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232,0,5275.503745,Bullish Low Variance,5819.081901,5724.311315,Bullish,Bullish
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050,0,5280.514303,Bullish Low Variance,5822.969325,5731.147905,Bullish,Bullish


In [91]:
# Define initial exposure based on Adjusted_Market_Regime
exposure_mapping = {
    'Bullish Low Variance': 2.0,
    'Bearish Low Variance': 1.0,
    'Bullish High Variance': 1.0,
    'Bearish High Variance': 0.0
}
data['Portfolio_Exposure'] = data['Market_Regime'].map(exposure_mapping).fillna(1.0)  # Default exposure is 1.0 if regime is NaN

# Adjust exposure based on 30-Day and 60-Day Indicators
for index, row in data.iterrows():
    if row['Portfolio_Exposure'] == 2.0:
        if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
            data.at[index, 'Portfolio_Exposure'] = 1.0
        elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
            data.at[index, 'Portfolio_Exposure'] = 1.5
        elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
            data.at[index, 'Portfolio_Exposure'] = 1.5
            
# Adjust exposure based on 30-Day and 60-Day Indicators for exposure = 1.0 and Bearish Low Variance regime
for index, row in data.iterrows():
    if row['Portfolio_Exposure'] == 1.0 and row['Market_Regime'] == 'Bearish Low Variance':
        if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
            data.at[index, 'Portfolio_Exposure'] = 0.0
        elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
            data.at[index, 'Portfolio_Exposure'] = 1.0
        elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
            data.at[index, 'Portfolio_Exposure'] = 1.0

In [92]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime,250_TMA,Market_Regime,30_TMA,60_TMA,30_Day_Indicator,60_Day_Indicator,Portfolio_Exposure
Date,,,,,,,,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0,16.850000,Bullish Low Variance,NaN,NaN,Bearish,Bearish,1.0
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0,16.870000,Bullish Low Variance,16.850000,16.850000,Bullish,Bullish,2.0
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0,16.886667,Bullish Low Variance,16.870000,16.870000,Bullish,Bullish,2.0
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0,16.905000,Bullish Low Variance,16.886667,16.886667,Bullish,Bullish,2.0
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0,16.918800,Bullish Low Variance,16.905000,16.905000,Bullish,Bullish,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0,5270.483643,Bullish Low Variance,5815.639277,5717.656303,Bullish,Bullish,2.0
2024-11-13 00:00:00+00:00,5985.379883,5985.379883,6008.189941,5965.910156,5985.750000,4220180000,0.000232,0,5275.503745,Bullish Low Variance,5819.081901,5724.311315,Bullish,Bullish,2.0
2024-11-14 00:00:00+00:00,5949.169922,5949.169922,5993.879883,5942.279785,5989.680176,4184570000,-0.006050,0,5280.514303,Bullish Low Variance,5822.969325,5731.147905,Bullish,Bullish,2.0


In [93]:
last_day_250_TMA = data['250_TMA'][-1]
last_day_adjusted_close = data['Adj Close'][-1]

In [94]:
# Define the four market regimes for 250 TMA
next_day_conditions = [
    
    predicted_most_likely_regime == 1 and last_day_adjusted_close < last_day_250_TMA,
    predicted_most_likely_regime == 1 and last_day_adjusted_close >= last_day_250_TMA,
    predicted_most_likely_regime == 0 and last_day_adjusted_close < last_day_250_TMA,
    predicted_most_likely_regime == 0 and last_day_adjusted_close >= last_day_250_TMA,
    
]
next_day_choices = [
    'Bearish High Variance',
    'Bullish High Variance',
    'Bearish Low Variance',
    'Bullish Low Variance'
]

In [95]:
next_day_market_regime = np.select(next_day_conditions, next_day_choices, default='Unknown')
next_day_market_regime

array('Bullish Low Variance', dtype='<U21')

In [96]:
next_day_30_TMA = data['30_TMA'][-1]
next_day_60_TMA = data['60_TMA'][-1]
next_30_Day_Indicator = np.where(last_day_adjusted_close > next_day_30_TMA, 'Bullish', 'Bearish')
next_60_Day_Indicator = np.where(last_day_adjusted_close > next_day_60_TMA, 'Bullish', 'Bearish')

In [97]:
print(next_day_30_TMA)
print(next_day_60_TMA)
print(next_30_Day_Indicator)
print(next_60_Day_Indicator)

5831.029639756945
5745.131529405381
Bullish
Bullish


In [98]:
# Example value of next_day_market_regime as a numpy array
next_day_market_regime = np.array(['Bullish Low Variance'])  # Single value stored as an array

# Define the exposure mapping
exposure_mapping = {
    'Bullish Low Variance': 2.0,
    'Bearish Low Variance': 1.0,
    'Bullish High Variance': 1.0,
    'Bearish High Variance': 0.0
}

# Extract the single value from the numpy array
next_day_market_regime = next_day_market_regime.item()  # Or use next_day_market_regime[0]

# Map the regime to exposure
next_day_exposure = exposure_mapping[next_day_market_regime]

print(f"Next Day Market Regime: {next_day_market_regime}")
print(f"Exposure: {next_day_exposure}")

Next Day Market Regime: Bullish Low Variance
Exposure: 2.0


In [99]:
# Adjust exposure based on 30-Day and 60-Day Indicators
for index, row in data.iterrows():
    if next_day_exposure == 2.0:
        if next_30_Day_Indicator == 'Bearish' and next_60_Day_Indicator == 'Bearish':
            next_day_exposure = 1.0  # Fix assignment here
        elif next_30_Day_Indicator == 'Bullish' and next_60_Day_Indicator == 'Bearish':
            next_day_exposure = 1.5
        elif next_30_Day_Indicator == 'Bearish' and next_60_Day_Indicator == 'Bullish':
            next_day_exposure = 1.5

# Adjust exposure based on 30-Day and 60-Day Indicators for exposure = 1.0 and Bearish Low Variance regime
for index, row in data.iterrows():
    if next_day_exposure == 1.0 and next_day_market_regime == 'Bearish Low Variance':
        if next_30_Day_Indicator == 'Bearish' and next_60_Day_Indicator == 'Bearish':
            next_day_exposure = 0.0
        elif next_30_Day_Indicator == 'Bullish' and next_60_Day_Indicator == 'Bearish':
            next_day_exposure = 1.0
        elif next_30_Day_Indicator == 'Bearish' and next_60_Day_Indicator == 'Bullish':
            next_day_exposure = 1.0

In [100]:
next_day_exposure

2.0

In [101]:
#Telegram Messenger
# Telegram Bot API token and Channel ID
bot_token = '7328648943:AAH3gHyGf2xgjxBfzPd05F_7IagASgs-Dj0'
channel_id = '-1002309744206'

# Initialize the message variable each time the code runs with bold header
message = "<b>Your Daily Portfolio Exposure Update</b>\n\n"  # Reset message here
# labels = ["Tomorrow's Predicted Market Regime"]

# Parse the predicted_date and add 1 day
formatted_date = (datetime.strptime(predicted_date, '%Y-%m-%d') + timedelta(days=0)).strftime('%m/%d/%Y')

# Remove leading zeros from month and day
formatted_date = formatted_date.lstrip("0").replace("/0", "/")

print(f"Formatted Date: {formatted_date}")

# Add the labeled message for each row with line breaks for better formatting
message += f"<u>Tomorrow's Predicted Market Regime</u>\n"
message += f"<i>Date</i>: {formatted_date}\n"
message += f"<i>Adjusted Market Regime</i>: {next_day_market_regime}\n"
message += f"<i>Portfolio Exposure</i>: {next_day_exposure * 100:.0f}%\n\n" # Format Portfolio_Exposure as a percentage with 2 decimal places

# Telegram API URL
api_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
# Payload to send with HTML formatting enabled
payload = {
    'chat_id': channel_id,
    'text': message,  # Combine with the rest of your message
    'parse_mode': 'HTML'  # Enables HTML for bold formatting
}
# Send the request
response = requests.post(api_url, json=payload)
# Check the response
if response.status_code == 200:
    print('Message sent successfully!')
else:
    print(f'Failed to send message. Error: {response.text}')


Formatted Date: 11/19/2024
Message sent successfully!


In [102]:
# # Ensure 'predicted_date' is a valid date object or convert it
# if not isinstance(predicted_date, pd.Timestamp):
#     predicted_date = pd.Timestamp(predicted_date)

# # Check if 'predicted_date' already exists in the DataFrame
# if predicted_date not in data.index:
#     # Create a new row with NaN for all columns except 'Recursive_Predictions'
#     new_row = {'Recursive_Predictions': most_likely_regime, 'Recursive_Predicted_Regime_Label': None}

#     # Append the row using the Date index
#     data.loc[predicted_date] = pd.Series(new_row)

# # Access the last row using iloc[-1] and update the 'Recursive_Predictions' column
# data.iloc[-1, data.columns.get_loc('Vol_Regime')] = most_likely_regime